# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','EarlyLesionLearning.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.WARNING)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

Plot group learning curve

In [ ]:
def plot_learning_curve(root, ax, profile, TaskParamToPlot, animalList=None, NSession=30, plotAnimal=True,**kwargs):
    assert isinstance(TaskParamToPlot,str)
    #learining curve
    if animalList is None:
        animalList= batch_get_animal_list(root,profile)
    if plotAnimal:
        colors=get_colors(len(animalList),colormap='cool')
    
    data=np.ones((NSession,len(animalList)))*np.nan
    x=np.arange(1,NSession+1)
    
    badAnimals=[]
    for i,animal in enumerate(animalList):
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot, profile=profile)
        if len(out) < NSession:
            badAnimals.append(i)
            continue
        data[:,i]=out[:NSession]
        if plotAnimal:
            ax.plot(x, out[:NSession],color=colors[i],lw=.5,alpha=.5,zorder=0)
            
    
    data=np.delete(data,badAnimals,axis=1)
    
    y=np.nanmedian(data,axis=1)
    Gline=ax.plot(x, y,zorder=4,**kwargs)
    if not plotAnimal:
        ax.fill_between(x,y1=np.nanpercentile(data,25,axis=1),y2=np.nanpercentile(data,75,axis=1),
                        fc='xkcd:black',ec=None,alpha=.4,zorder=3)

    ax.set_xlim([x[0]-1,x[-1]+1])
    xtick=[1]
    for i in range(1,NSession+1):
        if i%10==0:
            xtick.append(i)
    ax.set_xticks(xtick)
    ax.spines['bottom'].set_bounds(x[0],x[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
#     ax.set_ylim([0,10])
#     ax.set_yticks([0,7,10])
#     ax.set_ylabel(TaskParamToPlot)
    logging.warning(f'Removed due to not enough sessions:{[animal for i,animal in enumerate(animalList) if i in badAnimals]}')
    animalList=[animal for i,animal in enumerate(animalList) if i not in badAnimals]
    return data, animalList

In [ ]:
if "__file__" not in dir():

    profile={'Type':'Good',
                      'option': ['not used', 'AsymmetricLesion'],
                      'rewardType':'Progressive',
                      'initialSpeed':['10'],
                      'Speed':'10',
                      'Tag':['Early-Lesion_DS','Early-Lesion_DS-NoHab']}

    TaskParamToPlot='median entrance time (sec)'
    NSession =30
    
    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111);
    
    data, animalList=plot_learning_curve(root, ax, profile, TaskParamToPlot, NSession=NSession)
    plt.show()

---

plotting the trajectories of example sessions

In [ ]:
def plot_trajectories(data,ax):
    posDict=data.position
    time=data.timeTreadmill #align on camera
    
    seqTrials,_=sequentialTrials(data)._compute_sequential_trials()
    
    Colors=[]
    for trial in posDict:
        color="steelblue"
        alpha=1
        zorder=1
        if trial not in seqTrials:
            color="salmon"
            alpha=1
            zorder=0
        Colors.append(color)
        ax.plot(time[trial][:data.stopFrame[trial]], posDict[trial][:data.stopFrame[trial]],
               color=color, lw=.5, alpha=alpha, zorder=zorder)
            
    
    return np.array(Colors)



def plot_trajectories_and_distributions(root, ax, session, showText=True):
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    color=plot_trajectories(data,ax=ax)
    
    position=get_positions_array_beginning(data,onlyGood=False,raw=False)
    position=position.T
    
    
    if showText:
        ax.set_xlim([-1,11])
        ax.set_xticks([0,7,11])
        ax.set_ylim([0,90])
        ax.set_yticks([0,45,90])
        ax.set_yticklabels([0,'',90])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_bounds(0,11)
        ax.set_xlabel('Trial time (s)',labelpad=0)
        ax.set_ylabel('Position (cm)',labelpad=0)
    else:
        ax.set_xlim([-1,11])
        ax.set_ylim([0,90])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.tick_params(bottom=False, top=False, left=False, right=False,
                      labelbottom=False, labeltop=False, labelleft=False, labelright=False)


In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(4,5))
    ax=fig.add_subplot(111)
    
    
    session='Rat312_2019_01_16_07_59'

    
    plot_trajectories_and_distributions(root, ax, session, showText=False)

---

Front-Staying Trials

In [ ]:
def _perc_frontal_trials(data, xFront=30):
    data.position_correction()
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)
    timePoint=5 #s

    frontTrial=[]
    for trial,pos in data.position.items():
        _end=int(min([timePoint*cs,len(pos)-1]))
        
        #going backward in the first 5s
        if np.sum(pos[trialStart:_end]>xFront)>.1*cs:
            continue

        #The trial IS indeed Frontal
        frontTrial.append(trial)

    return (len(frontTrial)/data.nTrial)*100

In [ ]:
if "__file__" not in dir():
    session='Rat238_2018_06_19_10_16'
    data=Data(root,session[:6],session,redoPreprocess=False)
    
    print(_perc_frontal_trials(data))

---

plot the Frontal trial correlation wit hlesion

In [ ]:
def behav_lesion_size_correlation(root,ax,PerfParamToPlot,AnimalProfile, color,
                                  animalList=None, Ndays=slice(24,30), **kwargs):

    if animalList is None:
        animalList=batch_get_animal_list(root,AnimalProfile)

    size=[]
    behav=[]
    animals=[]
    for animal in animalList:
        try:
            i=HistologyExcel(NAS_PATH,animal)
        except AssertionError as e:
            logging.warning(repr(e))
            continue
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        

        res=animal_learning_stats(root, animal, PerfParamToPlot, profile=AnimalProfile,
                                  sessionSlice=Ndays, redo=False)
        
        try:
            t1=np.nanmedian(res)
            t3=i.lesion_size()
        except Exception as e:
            logging.warning(repr(e))
            continue
        behav.append(t1)
        size.append(t3)
        animals.append(animal)
    
        #PLOTTING
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=6, c=color[tag], label=tag, **kwargs);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'{PerfParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markerfacecolor='xkcd:gray', marker='o',markeredgewidth=.1,
                                       markeredgecolor='xkcd:black', markersize=3, label='Late')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:black', marker='D',markeredgewidth=.1,
                                       markerfacecolor='xkcd:gray', markersize=3, label='Early')

    leg=ax.legend(handles=[r_marker,b_marker],loc=(0,-.02),mode=None,ncol=3,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False,handletextpad=0,columnspacing=1)
    return leg

In [ ]:
if "__file__" not in dir():
    
    #Define desired profile dictionary
    AnimalProfile={'Type':'Good',
         'option': ['not used'],
         'rewardType':'Progressive',
         'initialSpeed':['10'],
         'Speed':'10',
         'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS']
                  }
    PerfParamToPlot="Forward Running Speed"
    color={'DS':'m','DMS':'b', 'DLS':'r', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    behav, size, animals=behav_lesion_size_correlation(root,ax,PerfParamToPlot,profile4,color, marker='v')
    
    r,p=stats.pearsonr(size, behav)
    print(r,p)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='k'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'
    colorSig='goldenrod'

    
    #===============================================
    
    # GRID 1 PARAMS
    
    profilecontrol1={'Type':'Good',
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Control', 'Control-Sharp']}
    
    profileEarlyDLS1={'Type':'Good',
                      'option': ['not used', 'AsymmetricLesion'],
                      'rewardType':'Progressive',
                      'initialSpeed':['10'],
                      'Speed':'10',
                      'Tag':['Early-Lesion_DLS']}

    profileEarlyDMS1={'Type':'Good',
                      'option': ['not used', 'AsymmetricLesion'],
                      'rewardType':'Progressive',
                      'initialSpeed':['10'],
                      'Speed':'10',
                      'Tag':['Early-Lesion_DMS','Early-Lesion_DMS-NoHab']}

    profileEarlyDS1= {'Type':'Good',
                      'option': ['not used', 'AsymmetricLesion'],
                      'rewardType':'Progressive',
                      'initialSpeed':['10'],
                      'Speed':'10',
                      'Tag':['Early-Lesion_DS','Early-Lesion_DS-NoHab']}
    
    TaskParamToPlot1='median entrance time (sec)'
    NSession1 =30
    
    TaskParamToPlot2="Motor Sequence Score"
    TaskParamToPlot3="Forward Running Speed"

    Profiles1=(profilecontrol1,profileEarlyDLS1,profileEarlyDMS1,profileEarlyDS1)
    TaskParamToPlots1=(TaskParamToPlot1,TaskParamToPlot2)#,TaskParamToPlot3)
    Ylabels1=('Entrance time (s)','% Routine','Speed (cm/s)')
    
    badAnimals1=['Rat115','Rat204','Rat233','Rat237','Rat238','Rat242']
    
    
    #===============================================
    
    # GRID 2&3 PARAMS
    
    session0Ex1 ='Rat233_2018_04_10_16_27'
    session30Ex1='Rat233_2018_05_23_14_55'
    session0Ex2 ='Rat238_2018_05_08_12_59'
    session30Ex2='Rat238_2018_06_19_10_16'
    
    
    #===============================================
    
    # GRID 4 PARAMS

    TaskParamToPlot4=_perc_frontal_trials.__name__
    
    profile4={'Type':'Good',
             'option': ['not used', 'AsymmetricLesion'],
             'rewardType':'Progressive',
             'initialSpeed':['10'],
             'Speed':'10',
             'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                            'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}

    #===============================================
    
    # GRID 5 PARAMS


    profile5={'Type': 'Good',
                 'rewardType': 'Progressive',
                 'option': ['not used', 'AsymmetricLesion'],
                 'initialSpeed': ['0', '10'],
                 'Speed': '10',
                 'Tag': ['Late-Lesion_DS-Sharp',
                  'Late-Lesion_DLS',
                  'Late-Lesion_DMS',
                  'Late-Lesion_DMS-Sharp',
                  'Late-Lesion_DS']}


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(7,3.5)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        

    ##########################################
    # 1: 
    gs1= fig.add_gridspec(nrows=2, ncols=4, left=0, bottom=.34, right=1, top=1, wspace=.1)
    
    Ymax1=(10.5,.95,95)
    axes1=[]
    for i,TaskParamToPlot in enumerate(TaskParamToPlots1):
        axes1.append(fig.add_subplot(gs1[i,0]))
        dCtrl,_=plot_learning_curve(root, axes1[-1], Profiles1[0], TaskParamToPlot, NSession=NSession1,color=CtrlColor)
        axes1[-1].set_ylabel(Ylabels1[i])
        
        axes1.append(fig.add_subplot(gs1[i,1]))
        dDLS,_=plot_learning_curve(root, axes1[-1], Profiles1[1], TaskParamToPlot, NSession=NSession1,color=DLSColor)
        axes1[-1].plot(np.arange(1,NSession1+1), np.nanmedian(dCtrl,axis=1),color=CtrlColor,zorder=3,lw=1)
        p=permtest(group1=dCtrl.T, group2=dDLS.T, nIterations=10000)
        p.plotSignificant(ax=axes1[-1],y=Ymax1[i],color=colorSig,lw=2)
        
        axes1.append(fig.add_subplot(gs1[i,2]))
        dDMS,_=plot_learning_curve(root, axes1[-1], Profiles1[2], TaskParamToPlot, NSession=NSession1,color=DMSColor)
        axes1[-1].plot(np.arange(1,NSession1+1), np.nanmedian(dCtrl,axis=1),color=CtrlColor,zorder=3,lw=1)
        p=permtest(group1=dCtrl.T, group2=dDMS.T, nIterations=10000)
        p.plotSignificant(ax=axes1[-1],y=Ymax1[i],color=colorSig,lw=2)
        
        axes1.append(fig.add_subplot(gs1[i,3]))
        dDS,_=plot_learning_curve(root, axes1[-1], Profiles1[3], TaskParamToPlot, NSession=NSession1,color=DSColor)
        axes1[-1].plot(np.arange(1,NSession1+1), np.nanmedian(dCtrl,axis=1),color=CtrlColor,zorder=3,lw=1)
        p=permtest(group1=dCtrl.T, group2=dDS.T, nIterations=10000)
        p.plotSignificant(ax=axes1[-1],y=Ymax1[i],color=colorSig,lw=2)
        
        
        
    axes1[3].annotate('', xy=(30, 1.6), xytext=(27, 3.5),
            arrowprops=dict(facecolor='k',arrowstyle='->',shrinkB=.1),fontsize=5
            )
    axes1[3].text(24,3.5,f'{len(badAnimals1)} animals',fontsize=5,ha='left',va='bottom')
        
    for ax in axes1[:4]:
        ax.set_ylim([1,11])
        ax.spines['left'].set_bounds(1,11)
        ax.set_yticks(np.arange(1,12,2))
        ax.set_yticklabels([])
        ax.set_xticklabels([])

    for ax in axes1[4:8]:
        ax.set_ylim([0,1])
        ax.set_yticks(np.arange(0,1.01,.2))
        ax.set_yticklabels([])
        ax.set_xticklabels([])

    axes1[0].set_title(f'Control ($n={dCtrl.shape[1]}$)',pad=2,fontsize='x-small')
    axes1[1].set_title(f'DLS ($n={dDLS.shape[1]}$)',pad=2,fontsize='x-small')
    axes1[2].set_title(f'DMS ($n={dDMS.shape[1]}$)',pad=2,fontsize='x-small')
    axes1[3].set_title(f'DS ($n={dDS.shape[1]}$)' ,pad=2,fontsize='x-small')
    axes1[-4].set_xlabel('Learning session #')
    axes1[-4].set_xticklabels([1,10,20,30])
    axes1[-4].set_yticklabels(np.arange(0,101,20))
    axes1[0].set_yticklabels(np.arange(1,12,2))
    
    
    
    
    ##########################################
    # 2:DS Ex1
    gs2= fig.add_gridspec(nrows=1, ncols=2, left=0.1, bottom=-.12, right=.35, top=.03)
    ax2= fig.add_subplot(gs2[0])
    ax2_= fig.add_subplot(gs2[1])
    
    plot_trajectories_and_distributions(root, ax2, session0Ex1, showText=True)
    plot_trajectories_and_distributions(root, ax2_, session30Ex1, showText=False)
    
    ax2.text(.5,0,f'R{session0Ex1[3:6]} (#1)',fontsize='xx-small',ha='center',va='bottom',transform=ax2.transAxes)
    ax2_.text(.5,0,f'R{session30Ex1[3:6]} (#30)',fontsize='xx-small',ha='center',va='bottom',transform=ax2_.transAxes)
    
    
    
    
    ##########################################
    # 3:DS Ex2
    gs3= fig.add_gridspec(nrows=1, ncols=2, left=.1, bottom=.05, right=.35, top=.2)
    ax3= fig.add_subplot(gs3[0])
    ax3_= fig.add_subplot(gs3[1])
    
    plot_trajectories_and_distributions(root, ax3, session0Ex2, showText=False)
    plot_trajectories_and_distributions(root, ax3_, session30Ex2, showText=False)

    ax3.text(.5,0,f'R{session0Ex2[3:6]} (#1)',fontsize='xx-small',ha='center',va='bottom',transform=ax3.transAxes)
    ax3_.text(.5,0,f'R{session30Ex2[3:6]} (#30)',fontsize='xx-small',ha='center',va='bottom',transform=ax3_.transAxes)

    
    
    ##########################################
    # 4:DS lesion vs Front
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=.43, bottom=-.12, right=.63, top=.2)
    ax4= fig.add_subplot(gs4[0])
    
    behav4, size4, animals4=behav_lesion_size_correlation(root,ax4,TaskParamToPlot4,profile4,ColorCode,
                                                          marker='o',edgecolors='none',linewidths=.1,alpha=.8)
    
    for animal in badAnimals1:
        i=animals4.index(animal)
        ax4.scatter(size4[i], behav4[i], s=6,
                    color=DSColor, marker='D', edgecolor='none',linewidths=.1)

        
    ax4.set_ylim([-5,90])
    ax4.spines['left'].set_bounds(0,90)
    ax4.set_ylabel('% Frontal trials',labelpad=0)
    ax4.set_yticks([0,25,50,90])


    
    
    ##########################################
    #5: % All rats corrlation
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.7, bottom=-.12, right=.9, top=.2)
    ax5= fig.add_subplot(gs5[0])
    
    behav5, size5, animals5   =behav_lesion_size_correlation(root,ax5,TaskParamToPlot3,profile4,ColorCode,Ndays=slice(-5,None),
                                                             marker='D',edgecolors='none',linewidths=.1,alpha=.8)
    behav5_, size5_, animals5_=behav_lesion_size_correlation(root,ax5,TaskParamToPlot3,profile5,ColorCode,Ndays=slice(-5,None),
                                                             marker='o',edgecolors='none',linewidths=.1,alpha=.8)
    
    r5,p5=stats.pearsonr(size5+size5_, behav5+behav5_)
    s=f'$n={len(animals5+animals5_)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r5)+'\n'+'$p=$'+'{}'.format(SciNote(p5))
    ax5.text(1,90,s, ha='right',va='top', fontsize=5)
    
    add_legend_for_lesion_types(ax5)
    
    ax5.set_ylim([20,90])
    ax5.set_ylabel('Speed (cm/s)')
    ax5.spines['left'].set_bounds(20,90)
    ax5.set_yticks([20,55,90])

    
    
    
        
    
    fig.align_ylabels([axes1[0],axes1[4]])
    ############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(axes1[0],ax3,ax4,ax5)
    OFFX=np.array([.06]*len(AXES))
    OFFY=np.array([.03]*len(AXES))
    OFFX[[1]]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','EarlyLesionLearning.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()